In [ ]:
import torch
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
import plotly.graph_objects as go
import time
import os
device = 'cuda:0'
torch.manual_seed(0)
torch.cuda.is_available()

In [ ]:
Config = {
    "dx": 0.5,
    "bd": 20,
    "L": None,
    "N": None,
    "Z": None,
    "lr": 0.1,
    "mom": 0.0,
    "fast": False,
    "ion_ene": [0, 13.59844, 24.58738, 
                5.39171, 9.32269, 8.29803, 11.26030, 14.53414, 13.61806, 17.42282, 21.5646]
}
Molecule = {
"name": "Dopamine",
"element": [
  8,
  8,
  7,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1
],
"x": [
  -2.2392,
  -3.3557,
  4.4081,
  2.1628,
  0.704,
  2.9862,
  -0.0999,
  0.1434,
  -1.4642,
  -1.2209,
  -2.0247,
  2.5111,
  2.3332,
  2.849,
  2.6457,
  0.3315,
  0.7594,
  -1.6445,
  4.5468,
  4.7362,
  -3.1541,
  -3.5639
],
"y": [
  1.9626,
  -0.5612,
  0.2624,
  -0.0212,
  -0.1603,
  0.1008,
  0.9759,
  -1.4267,
  0.8456,
  -1.557,
  -0.4208,
  -0.8817,
  0.8564,
  -0.7888,
  0.9593,
  1.9659,
  -2.3195,
  -2.5496,
  1.0868,
  -0.5285,
  1.6866,
  -1.5074
],
"z": [
  0.0548,
  0.3868,
  0.3445,
  -0.6613,
  -0.385,
  0.6289,
  -0.2919,
  -0.2187,
  -0.0326,
  0.0407,
  0.1336,
  -1.2481,
  -1.2993,
  1.2541,
  1.2192,
  -0.4187,
  -0.2869,
  0.1686,
  -0.2388,
  -0.2089,
  0.2377,
  0.4721
]
}
Train = {
    "N": None,
    "parameter": None,
    "gradient": None,
    "density": None,
    "difference": []
}
Result = {
    "loss": None,
    "shape": None,
    "orbits": None,
    "orbits_e": None,
    "density": None,
    "Q_atom": None,
    "N_atom": None, 
    "Q_elec": None,
    "N_elec": None,
    "energy": None,
    "dipole": None
}

In [ ]:
# Visualize Function
def visualize(func, thres=1e-6):
    verts, faces, _, _ = measure.marching_cubes(func, thres, spacing=(0.1, 0.1, 0.1))
    intensity = np.linalg.norm(verts, axis=1)

    fig = go.Figure(data=[go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                                    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                                    intensity=intensity,
                                    colorscale='Agsunset',
                                    opacity=1.0)])

    fig.update_layout(scene=dict(xaxis=dict(visible=False),
                                 yaxis=dict(visible=False),
                                 zaxis=dict(visible=False),
                                 bgcolor='rgb(0, 0, 0)'),
                      margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [ ]:
# Density Functional Theory
def calculate(config, molecule, train, result):
    t_start = time.time()
    
    # Config
    Atom = []
    Qm = np.zeros((3,2))
    for i in range(len(molecule["element"])):
        Atom.append([molecule["element"][i], np.array([molecule["x"][i], molecule["y"][i], molecule["z"][i]]) * 1.889726125])
        Qm[:,0] = np.maximum(Qm[:,0], Atom[i][1])
        Qm[:,1] = np.minimum(Qm[:,1], Atom[i][1])
    for a in Atom:
        a[1] -= np.mean(Qm, axis=1)
    dx = config["dx"]
    bd = config["bd"]
    L = config["L"]
    N = config["N"]
    Z = config["Z"]
    lr = config["lr"]
    mom = config["mom"]
    fast = config["fast"]
    ion_ene = config["ion_ene"]
    if L is None:
        L = (Qm[:,0]-Qm[:,1]+2*bd)
    if N is None:
        N = np.rint(L/dx).astype('int32')
    if Z is None:
        Z = sum(molecule["element"])

    # Previous Density
    Q = np.zeros((3, N[0], N[1], N[2]))
    Q[0,:,:,:] = np.linspace(-L[0]/2, L[0]/2, N[0])[:, np.newaxis, np.newaxis]
    Q[1,:,:,:] = np.linspace(-L[1]/2, L[1]/2, N[1])[np.newaxis, :, np.newaxis]
    Q[2,:,:,:] = np.linspace(-L[2]/2, L[2]/2, N[2])[np.newaxis, np.newaxis, :]
    if len(train["difference"]) == 0:
        ND0 = 0
        for a in Atom:
            Za = a[0]
            Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
            Ra = np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0))
            Ia = ion_ene[Za] / 27.211
            ND0 += (Za*np.sqrt(512*(Ia**3))) / (8*np.pi) * np.exp(-np.sqrt(8*Ia)*Ra)
        ND0 = ND0.reshape(N[0]*N[1]*N[2])
        train["parameter"] = ND0
        train["gradient"] = 0
        train["density"] = ND0
        train["N"] = N
        print("L:", L, "N:", N)
        print("Initial:")
        visualize(train["parameter"].reshape(N), 1e-1)
    if not np.array_equal(train["N"], N):
        train["parameter"] = resize(train["parameter"].reshape(train["N"]), N).reshape(N[0]*N[1]*N[2])
        train["gradient"] = resize(train["gradient"].reshape(train["N"]), N).reshape(N[0]*N[1]*N[2])
        train["density"] = resize(train["density"].reshape(train["N"]), N).reshape(N[0]*N[1]*N[2])
        train["N"] = N
        print("L:", L, "N:", N)
        print("Resize:")
        visualize(train["parameter"].reshape(N), 1e-1)
    Gra = mom * train["gradient"] + (train["parameter"] - train["density"])
    Par = train["parameter"] - lr * Gra
    Par *= sum(molecule["element"]) / np.sum(Par*(dx**3))
    ND0 = Par
    
    # Kinetic Energy
    D = [sp.sparse.spdiags(np.array([np.ones([N[i]]), -2*np.ones([N[i]]), np.ones([N[i]])]), 
                           np.array([-1,0,1]), N[i], N[i]) 
         for i in range(3)]
    Lap = sp.sparse.kronsum(sp.sparse.kronsum(D[2],D[1]), D[0]) / (dx**2)
    T = -1/2 * Lap

    # External Energy
    V_ext = 0
    for a in Atom:
        Za = a[0]
        Qa = a[1][:, np.newaxis, np.newaxis, np.newaxis]
        V_ext += -Za / (np.sqrt(np.sum((Q-Qa)*(Q-Qa), axis=0)) + 1e-6)
    V_ext = sp.sparse.diags(V_ext.reshape(N[0]*N[1]*N[2]))
    
    # Hartree Energy
    V_har0 = sp.sparse.linalg.cg(Lap, -4*np.pi*ND0)[0]
    V_har = sp.sparse.diags(V_har0)

    # Exchange-Correlation Energy
    V_xc0 = -np.cbrt(3/np.pi) * np.cbrt(ND0)
    V_xc = sp.sparse.diags(V_xc0)
    
    # Solve
    H = (T + V_ext + V_har + V_xc).tocoo()
    H = torch.sparse_coo_tensor(indices=torch.tensor(np.vstack([H.row, H.col])), values=torch.tensor(H.data), size=H.shape).to(device)
    if fast:
        H = H.float()
    fn = [2 for i in range(Z//2)]
    if Z % 2 == 1:
        fn.append(1)
    eigval, eigvec = torch.lobpcg(H, len(fn), largest=False)

    # Density
    orbits_e = eigval.detach().cpu().numpy()
    orbits = eigvec.T.detach().cpu().numpy()
    orbits = orbits / np.sqrt(np.sum(orbits*orbits*(dx**3), axis=1))[:, np.newaxis]
    ND = np.zeros(N[0]*N[1]*N[2], dtype=np.float32)
    for ne, orb in zip(fn, orbits):
        ND += ne*(orb**2)
    ND *= sum(molecule["element"]) / np.sum(ND*(dx**3))
    Dif = np.sum(np.abs(ND - ND0) * (dx**3))
    
    # Train
    train["parameter"] = Par
    train["gradient"] = Gra
    train["density"] = ND
    train["difference"].append(Dif)
    
    # Result
    if result["loss"] is None or Dif < result["loss"]:
        # Distribution
        QA = np.array([a[1] for a in Atom], dtype='float64').T
        NA = np.array([a[0] for a in Atom], dtype='float64')
        QE = Q.reshape((3, N[0]*N[1]*N[2]))
        NE = ND0 * (dx**3)
        
        # Energy
        EN = 0
        for ne, orb_e in zip(fn, orbits_e):
            EN += ne * orb_e
        EN -= np.sum((1./2.) * V_har0 * ND0 * (dx**3))
        EN -= np.sum(V_xc0 * ND0 * (dx**3))
        EN += (-3/4) * np.cbrt(3/np.pi) * np.sum(np.power(ND0, 4/3) * (dx**3))
        for i in range(len(Atom)):
            for j in range(i+1, len(Atom)):
                EN += Atom[i][0] * Atom[j][0] / np.linalg.norm(Atom[i][1]-Atom[j][1])

        # Dipole Moment
        DM = np.zeros(3)
        DM += np.sum(NA[np.newaxis, :]*QA, axis=1)
        DM += np.sum(-NE[np.newaxis, :]*QE, axis=1)

        # Result
        result["loss"] = Dif
        result["shape"] = N
        result["orbits"] = orbits
        result["orbits_e"] = orbits_e
        result["density"] = ND0
        result["Q_atom"] = QA
        result["N_atom"] = NA
        result["Q_elec"] = QE
        result["N_elec"] = NE
        result["energy"] = EN
        result["dipole"] = DM
    
    t_end = time.time()
    print("Iteration:", len(train["difference"]), " / Time:", t_end-t_start)
    print("Difference:", Dif)
    visualize(ND.reshape(N), 1e-1)

In [ ]:
Config["dx"] = 0.4
Config["lr"] = 0.1
prel = len(Train["difference"])
for i in range(100):
    calculate(Config, Molecule, Train, Result)
    if i>=10 and Train["difference"][i+prel]>Train["difference"][i+prel-10]:
        break        

In [ ]:
Config["lr"] = 0.01
prel = len(Train["difference"])
for i in range(100):
    calculate(Config, Molecule, Train, Result)
    if i>=10 and Train["difference"][i+prel]>Train["difference"][i+prel-10]:
        break        

In [ ]:
Config["lr"] = 0.001
prel = len(Train["difference"])
for i in range(100):
    calculate(Config, Molecule, Train, Result)
    if i>=10 and Train["difference"][i+prel]>Train["difference"][i+prel-10]:
        break        

In [ ]:
Config["lr"] = 0.0001
prel = len(Train["difference"])
for i in range(100):
    calculate(Config, Molecule, Train, Result)
    if i>=10 and Train["difference"][i+prel]>Train["difference"][i+prel-10]:
        break        

In [ ]:
np.savez('dopamine', **Result)

In [ ]:
visualize(Result["density"].reshape(Result["shape"]), 1e-3)

In [ ]:
for i in range(Result["orbits"].shape[0]):
    orb = Result["orbits"][i,:] * Result["orbits"][i,:]
    orb =  orb * np.sum(Result["density"]) / np.sum(orb)
    visualize(orb.reshape(Result["shape"]), 1e-1)

In [ ]:
i = 3
orb = Result["orbits"][i,:] * Result["orbits"][i,:]
print(np.sum(orb*(0.4**3)))

In [ ]:
Result

In [ ]:
Read = np.load('dopamine.npz')
for k in Read:
    Train[k] = Read[k]
Train["difference"] = Train["difference"].tolist()

In [ ]:
for k in Read:
    print(k)

In [ ]:
Read[""]

In [ ]:
Result

In [ ]:
visualize(Result["density"].reshape(Result["N"]), 1e-1)

In [ ]:
if save:
    np.savez(save, **Result)

In [ ]:
Result["eigenvalue"].shape

In [ ]:
Result["eigenvector"].shape

In [ ]:
Result = {
    "loss": 10,
    "orbits": None,
    "orbits_e": None,
    "Q_atom": None,
    "N_atom": None, 
    "Q_elec": None,
    "N_elec": None,
    "energy": None,
    "dipole": None
}

In [ ]:
np.savez('dopamine', **Result)

In [ ]:
def predict(config, molecule, result):
    # Config
    Atom = []
    Qm = np.zeros((3,2))
    for i in range(len(molecule["element"])):
        Atom.append([molecule["element"][i], np.array([molecule["x"][i], molecule["y"][i], molecule["z"][i]]) * 1.8897])
        Qm[:,0] = np.maximum(Qm[:,0], Atom[i][1])
        Qm[:,1] = np.minimum(Qm[:,1], Atom[i][1])
    for a in Atom:
        a[1] -= np.mean(Qm, axis=1)
    dx = config["dx"]
    bd = config["bd"]
    L = config["L"]
    N = config["N"]
    Z = config["Z"]
    mix = config["mix"]
    fast = config["fast"]
    ion_ene = config["ion_ene"]
    if L is None:
        L = (Qm[:,0]-Qm[:,1]+2*bd)
    if N is None:
        N = np.rint(L/dx).astype('int32')
    if Z is None:
        Z = sum(molecule["element"])
    
    # Distribution
    NE = result["priori"][-2] * (dx**3)
    QE = np.zeros((3, N[0], N[1], N[2]))
    QE[0,:,:,:] = np.linspace(-L[0]/2, L[0]/2, N[0])[:, np.newaxis, np.newaxis]
    QE[1,:,:,:] = np.linspace(-L[1]/2, L[1]/2, N[1])[np.newaxis, :, np.newaxis]
    QE[2,:,:,:] = np.linspace(-L[2]/2, L[2]/2, N[2])[np.newaxis, np.newaxis, :]
    QE = QE.reshape(3, N[0]*N[1]*N[2])
    NA = np.array([a[0] for a in Atom], dtype='float64')
    QA = np.array([a[1] for a in Atom], dtype='float64').T
    
    # Dipole Moment
    DM = np.zeros(3)
    DM += np.sum(-NE[np.newaxis, :]*QE, axis=1)
    DM += np.sum(NA[np.newaxis, :]*QA, axis=1)
    
    print(NE)
    print(QE)
    print(NA)
    print(QA)
    print(np.sum(-NE[np.newaxis, :]*QE, axis=1))
    print(np.sum(NA[np.newaxis, :]*QA, axis=1))
    
    # Result
    result["distribution"] = {"NE": NE, "QE": QE, "NA": NA, "QA": QA}
    result["dipole"] = DM
    

In [ ]:
# Load = np.load('dopamine.npz')
# for k in Load:
#     print(Load[k])
#     Result[k] = Load[k]
Load = np.load('dopamine.npz')
Result["N"] = Load["N"]
Result["parameter"] = Load["parameter"]
Result["gradient"] = Load["gradient"]
Result["density"] = Load["density"]
Result["difference"] = Load["difference"]

In [ ]:
for a in Result:
    print(a)

In [ ]:
print(Result["difference"])

In [ ]:
print(Result["eigenvector"])

In [ ]:
visualize(Result["density"].reshape(Result["N"]), 1e-1)

In [ ]:
Config["lr"] = 0.001
Config["dx"] = 0.4
prel = len(Result["difference"])
for i in range(100):
    calculate(Config, Molecule, Result)
    if i>=10 and Result["difference"][i+prel]>Result["difference"][i+prel-10]:
        break        

In [ ]:
Result["eigenvalue"] = Result["eigenvalue"].detach().cpu().numpy()
Result["eigenvector"] = Result["eigenvector"].detach().cpu().numpy()

In [ ]:
np.savez('dopamine', **Result)

In [ ]:
Temp = np.load('dopamine.npz')

In [ ]:
Temp["eigenvalue"]

In [ ]:
visualize(Temp["density"].reshape(Temp["N"]), 1e-1)

In [ ]:
Config["lr"] = 0.01
for i in range(len(Result["difference"]), len(Result["difference"])+100):
    calculate(Config, Molecule, Result)
    if i>=10 and Result["difference"][i]>Result["difference"][i-10]:
        break   

In [ ]:
Config = {
    "dx": 0.5,
    "bd": 20,
    "L": None,
    "N": None,
    "Z": None,
    "lr": 0.1,
    "mom": 0.0,
    "fast": False,
    "ion_ene": [0, 13.59844, 24.58738, 
                5.39171, 9.32269, 8.29803, 11.26030, 14.53414, 13.61806, 17.42282, 21.5646]
}

In [ ]:
Result["N"]

In [ ]:
Config["dx"] = 0.4
for i in range(len(Result["difference"]), len(Result["difference"])+100):
    calculate(Config, Molecule, Result)
    if i>=10 and Result["difference"][i]>Result["difference"][i-10]:
        break        

In [ ]:
print(np.prod(Config["N"]))

In [ ]:
print(Result["parameter"].shape[0])

In [ ]:
np.prod(Config["N"]) == Result["parameter"].shape[0]

In [ ]:
type(Result["parameter"].reshape(Config["N"]).shape)

In [ ]:
np.array_equal(Config["N"], Result["parameter"].reshape(Config["N"]).shape)

In [ ]:
print(Result["parameter"].reshape(Config["N"]).shape)

In [ ]:
A = Result["parameter"].reshape(Config["N"])

In [ ]:
B = resize(A, (220, 194, 180))

In [ ]:
visualize(B, 1e-1)

In [ ]:
visualize(Result["parameter"].reshape(Config["N"]), 1e-1)

In [ ]:
visualize(Result["density"].reshape(Config["N"]), 1e-2)

In [ ]:
visualize(Result["gradient"].reshape(Config["N"]), 1e-2)

In [ ]:
Config["lr"] = 0.01

In [ ]:
for i in range(100):
    calculate(Config, Molecule, Result)
    if Result["difference"][-1] < 1:
        break

In [ ]:
Config["lr"] = 0.001

In [ ]:
for i in range(100):
    calculate(Config, Molecule, Result)
    if Result["difference"][-1] < 1:
        break

In [ ]:
Config["lr"] = 0.0001

In [ ]:
for i in range(100):
    calculate(Config, Molecule, Result)
    if Result["difference"][-1] < 1:
        visualize(Result["density"].reshape([119, 95, 83]), 1e-3)
        break

In [ ]:
np.save('dopamine.npy', Result["parameter"][-1])

In [ ]:
visualize(Result["parameter"][-1].reshape([119, 95, 83]), 1e-3)

In [ ]:
visualize(Result["density"][-1].reshape([119, 95, 83]), 1e-3)

In [ ]:
for i in range(-10, 0):
    visualize(Result["gradient"][i].reshape([119, 95, 83]), 1e-3)

In [ ]:
predict(Config, Molecule, Result)

In [ ]:
if Save is not None:
    np.savez(Save, 
             NE=Result["distribution"]["NE"], QE=Result["distribution"]["QE"], 
             NA=Result["distribution"]["NA"], QA=Result["distribution"]["QA"])